In [144]:
import pandas as pd
import numpy as np
import xgboost as xgb
import sklearn

data = pd.read_parquet('train_data.parquet', engine='pyarrow')

In [136]:

y_data = data["target"]
x_data = data.drop('target', axis=1)
x_data

,id,sample_ml_new,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,...,feature1067,feature1068,feature1069,feature1070,feature1071,feature1072,feature1073,feature1074,feature1075,feature1076
0,0,1,1761,1759,85,105469,191,46,0,6,...,28913,48985,84264,12045,12107,12510,21126,28913,48985,84264
1,1,1,1761,1759,120,105610,144,71,0,135,...,0,0,0,0,0,0,0,0,0,0
2,2,1,890,1759,141,105227,191,11,0,0,...,55949,70006,113317,33735,31242,12691,48682,55949,70006,113317
5,5,1,1599,966,30,102441,191,8,0,0,...,0,6872,8530,0,0,0,0,0,6872,8530
6,6,1,1761,1759,85,104006,191,2,0,0,...,0,0,7642,0,0,0,0,0,0,7642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693036,693036,1,1761,1759,141,410,191,2,125,290,...,0,0,0,0,0,0,0,0,0,0
693038,693038,1,1761,1759,0,170,191,1,125,290,...,0,0,0,0,0,51714,0,0,0,0
693039,693039,1,1761,1759,0,168,191,1,125,290,...,0,0,0,0,0,51714,0,0,0,0
693040,693040,1,1761,1759,141,74,191,1,125,290,...,0,0,0,0,0,51714,0,0,0,0


In [115]:
xgb_data = xgb.DMatrix(x_data, label=y_data)

In [ ]:
params = {'max_depth': 5,
          'eta': 1,
          'objective': "binary:logistic",
          'nthread': -1,
          'eval_metric': ['auc', 'map']}
eval_list = [(xgb_data, "Everything")]
num_rounds = 100

In [143]:
bst = xgb.train(params, xgb_data, num_rounds, eval_list)

[0]	Everything-auc:0.69900	Everything-pre:0.28125
[1]	Everything-auc:0.72349	Everything-pre:0.53125
[2]	Everything-auc:0.73412	Everything-pre:0.50000
[3]	Everything-auc:0.74257	Everything-pre:0.59375
[4]	Everything-auc:0.74859	Everything-pre:0.62500
[5]	Everything-auc:0.75346	Everything-pre:0.68750
[6]	Everything-auc:0.75783	Everything-pre:0.56250
[7]	Everything-auc:0.76190	Everything-pre:0.59375
[8]	Everything-auc:0.76523	Everything-pre:0.62500
[9]	Everything-auc:0.76795	Everything-pre:0.71875
[10]	Everything-auc:0.76945	Everything-pre:0.71875
[11]	Everything-auc:0.77165	Everything-pre:0.71875
[12]	Everything-auc:0.77334	Everything-pre:0.78125
[13]	Everything-auc:0.77478	Everything-pre:0.84375
[14]	Everything-auc:0.77668	Everything-pre:0.81250
[15]	Everything-auc:0.77878	Everything-pre:0.78125
[16]	Everything-auc:0.78146	Everything-pre:0.78125
[17]	Everything-auc:0.78351	Everything-pre:0.81250
[18]	Everything-auc:0.78597	Everything-pre:0.84375
[19]	Everything-auc:0.78744	Everything-pr

In [157]:

n_right = list(y_data.loc[10:21])
preds = bst.predict(xgb.DMatrix(x_data))
preds = list(map(round, preds))
y = list(y_data)
"""for x in range(len(preds)):
    print(y, preds[x])"""
sklearn.metrics.recall_score(y, preds)

0.1642121163079247

In [107]:
data

,id,target,sample_ml_new,feature1,feature2,feature3,feature4,feature5,feature6,feature7,...,feature1067,feature1068,feature1069,feature1070,feature1071,feature1072,feature1073,feature1074,feature1075,feature1076
0,0,0,1,1761,1759,85,105469,191,46,0,...,28913,48985,84264,12045,12107,12510,21126,28913,48985,84264
1,1,0,1,1761,1759,120,105610,144,71,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,1,890,1759,141,105227,191,11,0,...,55949,70006,113317,33735,31242,12691,48682,55949,70006,113317
5,5,0,1,1599,966,30,102441,191,8,0,...,0,6872,8530,0,0,0,0,0,6872,8530
6,6,0,1,1761,1759,85,104006,191,2,0,...,0,0,7642,0,0,0,0,0,0,7642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693036,693036,0,1,1761,1759,141,410,191,2,125,...,0,0,0,0,0,0,0,0,0,0
693038,693038,0,1,1761,1759,0,170,191,1,125,...,0,0,0,0,0,51714,0,0,0,0
693039,693039,0,1,1761,1759,0,168,191,1,125,...,0,0,0,0,0,51714,0,0,0,0
693040,693040,0,1,1761,1759,141,74,191,1,125,...,0,0,0,0,0,51714,0,0,0,0


In [92]:
print(data[data.isna().any(axis=1)], sep="\n")


# index 0 feature756 nan
# index 315324 feature643, feature756 nan
# index 519615 feature756 nan

            id  target  sample_ml_new  feature1  feature2  feature3  feature4  \
0            0       0              1      1761      1759        85    105469   
1            1       0              1      1761      1759       120    105610   
2            2       0              1       890      1759       141    105227   
5            5       0              1      1599       966        30    102441   
6            6       0              1      1761      1759        85    104006   
...        ...     ...            ...       ...       ...       ...       ...   
693036  693036       0              1      1761      1759       141       410   
693038  693038       0              1      1761      1759         0       170   
693039  693039       0              1      1761      1759         0       168   
693040  693040       0              1      1761      1759       141        74   
693042  693042       1              1      1761      1759       141        28   

        feature5  feature6 

In [108]:
data.value_counts()

Series([], Name: count, dtype: int64)

In [22]:
pd.isnull(data).sum().value_counts()

0         1077
315324       1
519615       1
Name: count, dtype: int64

In [12]:
print(*list(data.columns), sep="\n")

id
target
sample_ml_new
feature1
feature2
feature3
feature4
feature5
feature6
feature7
feature8
feature9
feature10
feature11
feature12
feature13
feature14
feature15
feature16
feature17
feature18
feature19
feature20
feature21
feature22
feature23
feature24
feature25
feature26
feature27
feature28
feature29
feature30
feature31
feature32
feature33
feature34
feature35
feature36
feature37
feature38
feature39
feature40
feature41
feature42
feature43
feature44
feature45
feature46
feature47
feature48
feature49
feature50
feature51
feature52
feature53
feature54
feature55
feature56
feature57
feature58
feature59
feature60
feature61
feature62
feature63
feature64
feature65
feature66
feature67
feature68
feature69
feature70
feature71
feature72
feature73
feature74
feature75
feature76
feature77
feature78
feature79
feature80
feature81
feature82
feature83
feature84
feature85
feature86
feature87
feature88
feature89
feature90
feature91
feature92
feature93
feature94
feature95
feature96
feature97
feature98
featu

In [159]:
data['feature2'].value_counts()

feature2
1759    334027
0         4505
1         3242
2         2660
3         2174
         ...  
1177         7
1708         7
1711         6
1717         6
1518         5
Name: count, Length: 1728, dtype: int64

In [ ]:
data.corr()